In [1]:
# mogrify -path /home/izza/bismillah/dataset/train256 -resize "256x256^" -gravity center -crop 256x256+0+0 +repage *.jpeg
from __future__ import print_function

from resnet_helper import *
from resnet_structure import *

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/__init__.py", line 61, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
ImportError: numpy.core.multiarray failed to import


Failed to load the native TensorFlow runtime.

See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/get_started/os_setup.md#import_error

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [2]:
model = get_shallow_net(512, 10, K.image_dim_ordering())
optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = True)
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 512, 512)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 510, 510)  896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 32, 510, 510)  64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 510, 510)  0           batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [5]:
# X_train, y_train = load_dataset("splitted_train224", preprocess=True, categorical=True, zoom=False, shuffle=True)
# X_test, y_test = load_dataset("splitted_test224", preprocess=True, categorical=True, shuffle=True)

X_train, y_train = load_dataset("diabetic/data/train_tiff", preprocess=True, categorical=True, zoom=False, shuffle=True)
X_test, y_test = load_dataset("diabetic/data/test_tiff", preprocess=True, categorical=True, shuffle=True)

size[1 1 1 ..., 4 4 4]
dataset loaded total dataset 35126
size[1 1 1 ..., 4 4 4]
dataset loaded total dataset 53576


In [6]:
X_data = np.concatenate((X_train, X_test), axis=0)
y_data = np.concatenate((y_train, y_test), axis=0)

In [7]:
# metrics_param = ['accuracy', recall_metrics, precision_metrics, f1_metrics, true_positives]
metrics_param = ['accuracy']
resnet_model = get_modified_resnet() 
optimizer = SGD(lr = 0.0001, momentum = 0.9, decay = 0.9, nesterov = True)
resnet_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = metrics_param)

In [2]:
callbacks = [
    EarlyStopping(monitor='loss', patience=5, verbose=0),
    ModelCheckpoint(filepath="result/v2_rotate_00001_momentum_decay_extreme.hdf5", monitor='loss', verbose=0, save_best_only=True, save_weights_only=False)
]

for i in range(0, 100) :
    for degree in range(0, 360, 36) :
        X_train = rotate_dataset(X_data, degree)
        resnet_model.fit(X_data, y_data, nb_epoch=1, class_weight={0:1, 1:14, 2:7, 3:42, 4:46}, validation_split=0.2, callbacks=callbacks)                   
    resnet_model.save("v2_rotate_00001_momentum_decay_extreme.h5")

# CIFAR

In [5]:
from keras.datasets import cifar10
from keras.utils import *

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

In [3]:
resnet_model = get_modified_resnet(32, 10, K.image_dim_ordering(), origin=True, inception=True) #9562146 17041954
optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = True)
resnet_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
resnet_model.summary()

callbacks = [
#     EarlyStopping(monitor='loss', patience=5, verbose=0),
    ModelCheckpoint(filepath="../result/cifar.hdf5", monitor='loss', verbose=0, save_best_only=True, save_weights_only=False)
]

1
1
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, 32, 32)     0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 38, 38)     0           input_2[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 16, 16)    9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 16, 16)    128         conv1[0][0]                      
_______________________________________________________________________________________

In [8]:
resnet_model.fit(X_train, y_train, nb_epoch=100, callbacks=callbacks)                   
resnet_model.save("../cifar.h5")

Epoch 1/100


NotImplementedError: 
Apply node that caused the error: AveragePoolGrad{ds=(3, 3), ignore_border=True, st=(1, 1), padding=(1, 1), mode='average_exc_pad'}(Elemwise{Composite{Switch(i0, ((i1 * i2) + i3), ((i4 * i5) + i3))}}.0, IncSubtensor{InplaceInc;::, ::, :int64:, :int64:}.0)
Toposort index: 1106
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(32, 512, 1, 1), (32, 512, 1, 1)]
Inputs strides: [(2048, 4, 4, 4), (2048, 4, 4, 4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Add}[(0, 0)](CorrMM_gradInputs{half, (1, 1)}.0, CorrMM_gradInputs{half, (1, 1)}.0, AveragePoolGrad{ds=(3, 3), ignore_border=True, st=(1, 1), padding=(1, 1), mode='average_exc_pad'}.0, Split{2}.0, CorrMM_gradInputs{half, (1, 1)}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/usr/local/lib/python2.7/dist-packages/theano/gradient.py", line 1279, in access_grad_cache
    term = access_term_cache(node)[idx]
  

In [ ]:
scores = resnet_model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('Test score:', scores[0])
print('Test accuracy:', scores[1])

# ==============================================================

In [ ]:
callbacks = [
    EarlyStopping(monitor='loss', patience=5, verbose=0),
    ModelCheckpoint(filepath="result/v2_rotate_00001_momentum_decay_extreme.hdf5", monitor='loss', verbose=0, save_best_only=True, save_weights_only=False)
]

for i in range(0, 100) :
    for degree in range(0, 360, 36) :
        X_train = rotate_dataset(X_train, degree)
        resnet_model.fit(X_train, y_train, nb_epoch=1, class_weight={0:2, 1:15, 2:7, 3:41, 4:50}, callbacks=callbacks)
#         resnet_model.fit(X_train, y_train, nb_epoch=1, class_weight={0:1, 1:5, 2:7, 3:20, 4:50}, callbacks=callbacks)
                
#         result = resnet_model.predict(X_test, batch_size=32, verbose=0)
#         predict_classes, expected_classes = get_predicted_classes(result, y_test)
#         print(sklearn.metrics.confusion_matrix(expected_classes, predict_classes))

        if not_continue_epoch() :
            print("continue..")
        else :
            print("break")
            break
            
        
#     scores = resnet_model.evaluate(X_test, y_test, batch_size=32, verbose=0)
#     print('Test score:', scores[0])
#     print('Test accuracy:', scores[1])
#     result = resnet_model.predict(X_test, batch_size=32, verbose=0)
#     predict_classes, expected_classes = get_predicted_classes(result, y_test)
#     print("auc score " + str(sklearn.metrics.auc(expected_classes, predict_classes)))
#     print("f1 score " + str(sklearn.metrics.f1_score(expected_classes, predict_classes, average=None)))
#     print("recall score " + str(sklearn.metrics.recall_score(expected_classes, predict_classes, average=None)))
#     print("precision score " + str(sklearn.metrics.precision_score(expected_classes, predict_classes, average=None)))
#     print("cohen kappa score " + str(sklearn.metrics.cohen_kappa_score(expected_classes, predict_classes)))
    resnet_model.save("v2_rotate_00001_momentum_decay_extreme.h5")

Epoch 1/1
35126/35126 [==============================] - 737s - loss: 13.2500 - acc: 0.2421   
True
continue..
Epoch 1/1
35126/35126 [==============================] - 735s - loss: 12.5838 - acc: 0.2363   
True
continue..
Epoch 1/1
35126/35126 [==============================] - 734s - loss: 11.9984 - acc: 0.2271   
True
continue..
Epoch 1/1
35126/35126 [==============================] - 735s - loss: 11.5875 - acc: 0.2230   
True
continue..
Epoch 1/1
35126/35126 [==============================] - 735s - loss: 11.3487 - acc: 0.2225   
True
continue..
Epoch 1/1
 2368/35126 [=>............................] - ETA: 655s - loss: 11.1829 - acc: 0.2158